# **AAT DATA PROCESSING**

**Bibliotecas**

In [3]:
import re
from datetime import date
import pandas as pd

**Funcoes**

**Converter coordenadas Degrees, minutes, seconds para decimal degrees**

In [2]:
# degrees minutes seconds to decimal degress
def dms_to_dd(entry):
   graus, minutos, segundos, ponto_cardial = re.split('[°\'"]+', entry)
   #sin = -1 if str(ponto_cardial) == 'S' else 1
   convertido = (int(graus)+float(minutos)/60+float(segundos)/3600)
   return convertido

# degrees minutes to decimal degrees
def dm_to_dd(entry):
   graus, minutos, ponto_cardial = re.split('[°\']+', entry)
   #sin = -1 if str(ponto_cardial) == 'S' else 1
   convertido = (int(graus)+float(minutos)/60)
   return convertido

Em andamento...


In [ ]:
def count_tryp():
    dt['Tv'] = [tryp for n in dt['Trypanosoma'] if str(n) == "Tv": tryp = 1 else tryp = 0]
    dt["Tc"] = [tryp for n in dt["Trypanosoma"] if str(n) == "Tc": tryp = 1 else tryp = 0]
    dt["Tb"] = [tryp for n in dt["Trypanosoma"] if str(n) == "Tb": tryp = 1 else tryp = 0]
    dt['Total'] = dt['Tv']+dt['Tc']+dt['Tb']

**Adcionado a coluna Total para o numero de tripanossomas**

In [4]:
def total_count():
    dt['Total'] = dt['Tv']+dt['Tc']+dt['Tb']-dt['Tv_Tc']+dt['Tv_Tb']+dt['Tc_Tb']

**1. Carregando os dados**

In [5]:
dt = pd.read_excel("AAT_Manica_2014.xlsx")

**1.1 Processamento**

###### *A. converter as coordenadas*

In [17]:
# Grau minuto segundos

dt['Latitude'] = (dt['Latitude'].apply(dms_to_dd)) * -1
dt['Longitude'] = dt['Longitude'].apply(dms_to_dd)

In [ ]:
# Grau minutos

dt['Latitude'] = (dt['Latitude'].apply(dm_to_dd)) * -1
dt['Longitude'] = dt['Longitude'].apply(dm_to_dd)

######  *B. adicionar coluna total*

In [7]:
total_count()
dt.head()

,Source_file_name,Sheet_number,Year,Country,Province,District,Location,Longitude,Latitude,Date,...,AGE_Animals,SEX_Animals,PCV,Tv,Tc,Tb,Tv_Tc,Tv_Tb,Tc_Tb,Total
0,Mozambique_Repository/AAT/Manica/AAT_Manica_20...,1,2014,Mozambique,Manica,Vanduzi,Belas-Mozbife,NaN,NaN,2014-11-06,...,NaN,NaN,41,0,0,0,0,0,0,0
1,Mozambique_Repository/AAT/Manica/AAT_Manica_20...,1,2014,Mozambique,Manica,Vanduzi,Belas-Mozbife,NaN,NaN,2014-11-06,...,NaN,NaN,32,0,0,0,0,0,0,0
2,Mozambique_Repository/AAT/Manica/AAT_Manica_20...,1,2014,Mozambique,Manica,Vanduzi,Belas-Mozbife,NaN,NaN,2014-11-06,...,NaN,NaN,35,0,0,0,0,0,0,0
3,Mozambique_Repository/AAT/Manica/AAT_Manica_20...,1,2014,Mozambique,Manica,Vanduzi,Belas-Mozbife,NaN,NaN,2014-11-06,...,NaN,NaN,38,0,0,0,0,0,0,0
4,Mozambique_Repository/AAT/Manica/AAT_Manica_20...,1,2014,Mozambique,Manica,Vanduzi,Belas-Mozbife,NaN,NaN,2014-11-06,...,NaN,NaN,30,0,0,0,0,0,0,0


In [13]:
dt.columns

Index(['Source_fine_name', 'Sheet_number', 'Year', 'Country', 'Province',
       'District', 'Location', 'Longitude', 'Latitude', 'Date',
       'SPECIES_Animals', 'BREED_Animals', 'AGE_Animals', 'SEX_Animals', 'PCV',
       'Tv', 'Tc', 'Tb', 'Tv_Tc', 'Tv_Tb', 'Tc_Tb', 'Total'],
      dtype='object')

*C. agregar os dados por localidade*

In [35]:
# Aggregacao dos dados por localidade
agregado = dt.groupby(['Source_file_name','Province', 'District','Location']).agg({'Location':'count','PCV':'mean','Tv':'sum','Tc':'sum','Tb':'sum', 
'Tv_Tc':'sum', 'Tv_Tb':'sum', 'Tc_Tb':'sum', 'Total':'sum'})

agregado.rename(columns={'Location':'Sample_size'},inplace=True)
agregado.reset_index(inplace=True)

#agregado['Location'] = agregado.index.get_level_values('Location')

In [9]:
# Agregado PCV affected e PCV not affected
pcv_affected = dt.loc[dt['Total']==1]
pcv_not_affected = dt.loc[dt['Total']==0]

pcv_af_agg = pcv_affected.groupby('Location')[['PCV','Total']].mean()
pcv_no_agg = pcv_not_affected.groupby('Location')[['PCV','Total']].mean()

In [30]:
concatenado = pd.concat([pcv_no_agg, pcv_af_agg],axis=1)
concatenado.rename(columns={'PCV':'PCV_not_affected','PCV':'PCV_affected'},inplace=True)
concatenado.drop(['Total','Total'],axis=1,inplace=True)
concatenado.reset_index(inplace=True)
concatenado

,Location,PCV_affected,PCV_affected
0,Belas-Mozbife,35.961538,NaN
1,Mavonde-Mozbife,32.734694,28.5


*D. Calculo de prevalencia*

In [36]:
cols = ['Tv', 'Tc', 'Tb', 'Tv_Tc', 'Tv_Tb', 'Tc_Tb','Total']
ncols = ['TvPR', 'TcPR', 'TbPR', 'Tv_TcPR', 'Tv_TbPR', 'Tc_TbPR','TPR']

for c, nc in zip(cols,ncols): 
  agregado[nc] = agregado[c]/agregado['Sample_size']*100

# merging
agg = pd.merge(agregado,concatenado,how='outer',on='Location')

In [37]:
agg.columns

Index(['Source_file_name', 'Province', 'District', 'Location', 'Sample_size',
       'PCV', 'Tv', 'Tc', 'Tb', 'Tv_Tc', 'Tv_Tb', 'Tc_Tb', 'Total', 'TvPR',
       'TcPR', 'TbPR', 'Tv_TcPR', 'Tv_TbPR', 'Tc_TbPR', 'TPR', 'PCV_affected',
       'PCV_affected'],
      dtype='object')

*E. Salvar os dados*

In [38]:
# Resultado final
agg.head()


,Source_file_name,Province,District,Location,Sample_size,PCV,Tv,Tc,Tb,Tv_Tc,...,Total,TvPR,TcPR,TbPR,Tv_TcPR,Tv_TbPR,Tc_TbPR,TPR,PCV_affected,PCV_affected
0,Mozambique_Repository/AAT/Manica/AAT_Manica_20...,Manica,Vanduzi,Belas-Mozbife,26,35.961538,0,0,0,0,...,0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,35.961538,NaN
1,Mozambique_Repository/AAT/Manica/AAT_Manica_20...,Manica,Vanduzi,Mavonde-Mozbife,51,32.568627,0,2,0,0,...,2,0.0,3.921569,0.0,0.0,0.0,0.0,3.921569,32.734694,28.5


In [16]:
# Salvando todas tabelas numa multisheet excel

writer = pd.ExcelWriter('V_AAT_Manica_2014.xlsx', engine='xlsxwriter')

dt.to_excel(writer, sheet_name='Cleani')
agregado.to_excel(writer, sheet_name='agga')
writer.save()